In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this 0ㅣㅔ(by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
df_val = df_train[40016:];
df_train = df_train[:40000]
df_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
df_sample_submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [ ]:
df_val

In [ ]:
df_test

In [ ]:
df_sample_submission

In [ ]:
example_digit = df_train.loc[7].drop("label").to_numpy()
example_digit = example_digit.reshape(28,28)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(example_digit, cmap="gray")
plt.show()

print(df_train.loc[7]["label"])

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset) :
    def __init__(self, df_data) :
        self.x = torch.Tensor(df_data.drop("label",axis=1).values);
        self.y = torch.Tensor(df_data["label"].values);
    
    def __getitem__(self, idx) :
        return self.x[idx].reshape(1,28,28),self.y[idx];
    
    def __len__ (self) :
        return len(self.x);
    

In [ ]:
bs = 16;

train_dataset = CustomDataset(df_train)
train_dataloader = DataLoader(train_dataset, batch_size = bs, shuffle=True)
val_dataset = CustomDataset(df_val)
val_dataloader = DataLoader(val_dataset, batch_size = bs, shuffle=True)

In [ ]:
next(iter(train_dataloader))[0].shape

In [ ]:
from torch import nn

class HongNet(nn.Module) :
    def __init__(self) :
        super(HongNet, self).__init__()
        self.seq_module = nn.Sequential(
            nn.Conv2d(1,10,3,stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(10,10,3,stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(10,10,3,stride = 1, padding =1 ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(in_features = 10*13*13, out_features = 4096),
            nn.ReLU(),
            nn.Linear(in_features = 4096, out_features = 512),
            nn.ReLU(),
            nn.Linear(in_features = 512, out_features = 1),
            nn.ReLU()
        )
        
    def forward(self, x) :
        x = self.seq_module(x);
        #print("forward x value : ", x)
        x = x.reshape(bs,-1)
        #print(x.shape)
        return self.classifier(x);
    

In [ ]:
model = HongNet()
print(model)

In [ ]:
print(model.forward(next(iter(train_dataloader))[0]))

In [ ]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr= 1e-7)

In [ ]:
def train(dataloader, model, loss_fn, optimizer) :
    size = len(dataloader.dataset)
    for batch, (X, y), in enumerate(dataloader):
        pred = model.forward(X).view(16)
        #if( pred.count_nonzero() != 0) : 
        #    print(batch , " : prediction : " ,pred, "\n y : " , y);
            
        loss =loss_fn(pred, y)
        train_loss_log.append(loss);
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(batch%400==0) :
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            

In [ ]:
def test(dataloader, model, loss_fn) :
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss, correct =0, 0
    with torch.no_grad():
        for X, y in dataloader:
            
            pred = model.forward(X).view(16)
            ans = np.around(model.forward(X).detach().numpy().squeeze())
            #print("i predicted to ", pred," to ", ans, " and answer was ", y)
            val_loss += loss_fn(pred, y).item()
            correct+=torch.ne(torch.tensor(ans), y).sum().item()
    val_loss /= num_batches
    correct /= size
    print(f"val Error : \n Accuracy: { (100*correct) : >0.1f}% Avg loss : {val_loss :>8f}\n")

In [ ]:
epochs= 20
train_loss_log = []


In [ ]:
for t in range(epochs) :
    print(f"Epoch {t+1}\n-------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(val_dataloader, model, loss_fn)
    
print("Done!")